In [ ]:
source("../hub/map.r")
source("../hub/shortcuts.r")
source("../hub/run.r")

### 0 - Read exhaustive analysis

In [ ]:
oo_cat <- fread("1_run_fe.csv")
oo_lr <- fread("1_run_lr.csv") 

In [ ]:
cohorts <- fread("cohorts_ready.csv") 

In [ ]:
cohorts %>% 
 fi(grepl("Taxane", cohortGo)) %>% 
 gb(cohortGo) %>% 
 su(ct = n()) %>% 
 ar(desc(ct))

In [ ]:
cts <- 
cohorts %>% 
 tm(cohortGo, Samples = tot) %>% 
 unique() %>% 
 ar(desc(Samples))

### 1 - Combine and prepare data

- Make names cleaner
- Add multiple testing adjustment

In [ ]:
names <- c("nrBor" = "No Response", "nrDcb" = "No Durable Benefit")

In [ ]:
cohort_formatter <- function(i, samples){
  pieces <- strsplit(i, " ## ")[[1]]
  paste0(pieces[2], "\n", pieces[1], "\n", str_to_title(pieces[3]), " (N=", samples, ")")
}

In [ ]:
oo_base <- 
oo_lr %>% 
 fi(!grepl("sig_", x), y == "nrDcb") %>% 
 rj(oo_cat %>% fi(y == "nrDcb"), by = c("cohortGo", "y", "x")) %>% 
 lj(cts, by = "cohortGo") %>% 
 se(-data, -model) %>% 
 rw() %>% 
 mu(min_e = min(nr_e, r_e, nr_ne, r_ne), 
    y = names[[y]], 
    pval_go = ifelse(min_e < 5, fisher_pval, pval), ### use fisher exact factor 
    cohort = cohort_formatter(cohortGo, Samples),
    cohort2 = cohort_formatter(cohortGo, tot)
    ) %>% 
 ug() %>% 
 fi(tot_e > 7, tot_ne > 7) %>%  
 mu(z = est/se, 
    p_adj_by = p.adjust(pval_go, method = "BY"), 
    p_adj_fdr = p.adjust(pval_go, method = "fdr"))

- Order cohorts by sample size

In [ ]:
cohorts_order <- oo_base %>% se(cohortGo, Samples) %>% ar(desc(Samples)) %>% pu(cohortGo) %>% unique()

In [ ]:
oo_go <- 
oo_base %>% 
 mu(cohortGo = factor(cohortGo, levels = cohorts_order)) %>% 
 mu(Group = ifelse( grepl("rna", x), "RNA", "DNA"), 
    Group = ifelse( grepl("clin", x), "Clinical", Group)) %>% 
 mu(est = ifelse(est < -5, -4.5, est), 
    est = ifelse(est > 5, 4.5, est))

### 2 - Top No Response

In [ ]:
cohorts_remove <- c("Pan-Cancer ## Anti_AR ## contains", 
                    "Pan-Cancer ## Anti-AR ## exact")

In [ ]:
ds(20, 20)

In [ ]:
share <- 
oo_go %>%  
 relocate(cohortGo, cohort, cohort2) %>% 
 fi(pr_nr_given_e == 1, !cohortGo %in% cohorts_remove, pval_go < .01) %>% 
 tm(cohortGo, 
    cohort2, 
    y, 
    x, 
    Group, 
    `Fisher_pval_unadj` = pval_go,
    `NR_Given_E` = nr_e, 
    `R_Given_E` = r_e, 
    `NR_Given_NE` = nr_ne, 
    `R_Given_NE` = r_ne, 
    `NR%_Given_E` = paste0(100*round(pr_nr_given_e,2), "%"),
    `R%_Given_E` = paste0(100*round(1-pr_nr_given_e,2), "%"),
   ) %>%
 ar(desc(NR_Given_E)) %>% 
 fi(x != "hrd_brca1_gt_25") 

In [ ]:
o_dir <- "/data/experiments/notebooks/jusset/non_responders/output/share/"

In [ ]:
fwrite(share %>% fi(Group == "RNA"), paste0(o_dir, "non_response_rna.csv"))
fwrite(share %>% fi(Group == "DNA"), paste0(o_dir, "non_response_dna.csv"))

### 3 - Results share

- Share top results

In [ ]:
top_results <- 
oo_go %>% 
 gb(y, cohortGo, Group) %>% 
 mu(rk = row_number(pval)) %>% 
 fi(rk <= 10) %>% 
 ug() %>% 
 tm(cohort = cohortGo, y, x, covariate, 
    type, Group, est = round(est,2), 
    se = round(se, 2), 
    pval, 
    p_adj_by, p_adj_fdr,  
    Samples, rk) %>% 
 ar(cohort, y, Group, rk)

In [ ]:
fwrite(top_results, file = "top_results.csv")

### 4 - Volcanoes

In [ ]:
adj_maps <- 
list(
 "pval" = c("-Log10( P-value )", .001), 
 "p_adj_fdr" = c("-Log10( Adjusted P-value BH )",.1),
 "p_adj_by" = c("-Log10( Adjusted P-value BY )", .1) 
)

In [ ]:
plot <- 
function( 
 outcome = "No Response", 
 x_lab = "Log Odds Estimate (Higher values more non-response)", 
 p = "pval"){
 
 oo_go %>% pu(.data[[p]])   
    
 oo_go %>% 
  fi(y == outcome) %>% 
  ggplot(aes(x = est, y = -log10(.data[[p]]), color = Group)) + 
  geom_point() +
  facet_wrap(~ cohortGo, ncol = 7) +   
  xlim(-6,6) +
  xlab(x_lab) +
  ylab(adj_maps[[p]][1]) +
  geom_hline(yintercept = -log10(as.numeric(adj_maps[[p]][2])), 
             linetype = "dashed", color = "black") + 
  ggtitle( paste0( paste0(outcome, " vs 1,200 Clinical, Genomic, RNA features" ))) + 
  go_theme + 
  geom_text(data = top_results %>% fi(y == outcome), 
           aes(label = x), position = "jitter", size = 1.9, hjust = .5, vjust = 0)
}

In [ ]:
dcb <- plot(outcome = "No Durable Benefit", p = "pval")
dcb2 <- plot(outcome = "No Durable Benefit", p = "p_adj_fdr")
dcb3 <- plot(outcome = "No Durable Benefit", p = "p_adj_by")

In [ ]:
ds(16, 8)

In [ ]:
dcb2

In [ ]:
dcb3

In [ ]:
ggsave( "plts/bor_unadjusted.png" , bor, width = 16, height = 8)
ggsave( "plts/bor_bh.png" , bor2, width = 16, height = 8)
ggsave( "plts/bor_by.png" ,bor3, width = 16, height = 8)
ggsave( "plts/dcb_unadjusted.png" , dcb, width = 16, height = 8)
ggsave( "plts/dcb_bh.png" , dcb2, width = 16, height = 8)
ggsave( "plts/dcb_by.png" , dcb3, width = 16, height = 8)

### 3 - Results to share

In [ ]:
top_results <- 
oo_go %>% 
 gb(cohortGo, y, Group) %>% 
 mu(rk = row_number(desc(abs(z)))) %>% 
 fi(rk <= 5) %>% 
 ug() %>% 
 tm(cohort = cohortGo, y, x, covariate, Group, type, 
    est = round(est,2), se = round(se, 2), z = round(z,2), 
    pval, p_adj_bh = p_adj_fdr, p_adj_by, Samples, rk) %>% 
 ar(cohort, y, Group, rk)

In [ ]:
fwrite(top_results, "plts/top_results.csv")